# Exploration - Complete - 2 Customer Type:

The code below shows complete exploration for 2 tyes of customers (Reserved and Preemtive). Basic layout of code is that there ia a function exploration which inside it takes all values from user from main loop (at the end) and uses different functions(demand function, demand generation, probability calc., dual variable calc., etc.) to conduct exploration phase.


 ## Variable description or Code Appendix 
     
    ###### c     = total inventory at current time.
    ###### c0    = Inventory before or at start of booking period
    ###### T     = total booking time.
    ###### t     = time instances to divide booking time.
    ###### xt    = Unit of Booking time T one instance has.
    ###### K     = Total subphases in phase 1 (K x t = P1)
    ###### ki    = 1 Subphase in Phase 1 or 1 element in K range
    ###### P1    = total booking time of phase 1.
    ###### P2    = total percbooking time of phase 2,  P1+P2 = T, a phase can have many subphases.
    ###### p     = initial price list created after inputs of Highest and Lowest price taken form user.
    ###### df    = data frame for Exploration demand generated.
    ###### pt    = price for a particular instance, pt is in range p.
    ###### z     = dual variable at beginning, can be zero at 1st instance.
    ###### z_opt = z*, optimal dual variable.
    ###### p_opt = p*, optimal price in exploration.
    ###### delta = (δ) parameter used in exploration for narrowing price intervals near optimal price.
    ###### p_low = lower price limit when we narrow price intervals towards optimal price.
    ###### p_upp = upper price limit when we narrow price intervals towards optimal price.
    ###### num_steps = Total number of Price points wanted
    ###### customers_per_price = number of customers approaching per price
    ###### total_customers = 2*customers_per_price, total customers approaching for both combined prices in an instance
    ###### Customer_data = Empty List to store customer data
    ###### demand_probability    = demand function for a particular instance, probability of buying at price pt and instance t. 

In [ ]:
# importing libraries
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import math
import matplotlib.pyplot as plt

####################################################################################################################################################################################################
# Exploration (Phase - 1) Function:

def exploration(c,c0, T, unit_T, t, xt, ki, K, P1, p, p_2, z, total_customers, num_steps):
    
    customer_data = []
    customer_id = 1


    for _ in range(total_customers):
        # Randomly assign customer type: 1 (Reserved) or 2 (Preemptive)
        cust_type = np.random.choice([1, 2], p=[0.5, 0.5])  # 50% Reserved, 50% Preemptive)

        # Pick price from appropriate price list
        if cust_type == 1:
            pt = np.random.choice(p)
            base_prob = logistic_demand_1(pt, p)
        else:
            pt = np.random.choice(p_2)
            base_prob = logistic_demand_2(pt, p_2)


        # Add noise to demand probability - diff, for different customer types
        if cust_type == 1:
            noise = np.random.normal(0, 0.08)  # More stable behavior
        else:
            noise = np.random.normal(0, 0.265)  # More volatile behavior

        prob = np.clip(base_prob + noise, 0.01, 0.99)
        buy = int(np.random.rand() < prob)

        # Store data: [customer_id, price offered, bought?, customer_type]
        customer_data.append([customer_id, pt, buy, cust_type])
        customer_id += 1


    # Create DataFrame including customer_type
    df = pd.DataFrame(customer_data, columns=["customer_id", "price", "buy", "customer_type"])
    #print(df.head())
    print(df)
    df.to_csv("customer_demand_log.csv", index=False)
    print("Saved customer data to 'customer_demand_log.csv'")




    # Calculate total units sold from both types
    units_sold = df['buy'].sum()
    c = float(c) - units_sold
    if c < 0:
        c = 0
    print("\n", f"Units sold in this phase: {units_sold}")
    print(f"Remaining inventory after sales: {c}")




    # Compute empirical demand probability separately per customer type
    print("\n** Type 1: Reserved **")
    demand_probability_1 = compute_demand_probability(df[df['customer_type'] == 1], p)

    print("\n** Type 2: Preemtive **")
    demand_probability_2 = compute_demand_probability(df[df['customer_type'] == 2], p_2)




    # Revenue calculation separately per customer type    
    total_revenue_1, revenue_per_price_1 = calculate_revenue(df[df['customer_type'] == 1])
    total_revenue_2, revenue_per_price_2 = calculate_revenue(df[df['customer_type'] == 2])
    total_revenue = total_revenue_1 + total_revenue_2

    print("\n*** Revenue Calculation for Customer Type 1 (Reserve) ***")
    for pt in p:
        print(f"Price: {pt:.2f}, Revenue: {revenue_per_price_1.get(pt, 0):.2f}")
    print("Total Revenue- type 1:",total_revenue_1)
    print("\n*** Revenue Calculation for Customer Type 2 (Preemptive) ***")
    for pt in p_2:
        print(f"Price: {pt:.2f}, Revenue: {revenue_per_price_2.get(pt, 0):.2f}")
    print("Total Revenue- type 2:",total_revenue_2)
    print(f"\nTotal Revenue (Both Types): {total_revenue:.2f}")




    # Dual optimization - now needs to consider both customer types separately or combined
    # You can do separate dual variable calculations or a weighted combined one.
    # For simplicity, run separately and sum the dual objectives or take weighted average
    z_opt_1 = dual_objective(float(z), demand_probability_1, p, float(c))
    z_opt_2 = dual_objective(float(z), demand_probability_2, p_2, float(c))
    # Combine dual variables with weighted average by revenue or units sold; here equal weight:
    z_opt = (z_opt_1 + z_opt_2) / 2
    print("\n*** Optimal dual variable (z*) combined:", z_opt)




    # Optimal price calculation separately
    print("\n** Type 1: Reserved **")
    p_opt_1 = exploration_optimal_price(z_opt_1, demand_probability_1, p)

    print("\n** Type 2: Preemtive **")
    p_opt_2 = exploration_optimal_price(z_opt_2, demand_probability_2, p_2)



    # Narrow price intervals separately
    print("\n** Type 1: Reserved **")
    p_low_1, p_upp_1, new_price_list_1 = narrow_price_interval(p_opt_1, 
                                                               n=len(df[df['customer_type'] == 1]),
                                                                k=ki, epsilon=0.01, p=p, num_steps=num_steps,
                                                                global_min=context['min_price_1'], 
                                                                global_max=context['max_price_1'])
    
    print("\n** Type 2: Preemtive **")
    p_low_2, p_upp_2, new_price_list_2 = narrow_price_interval(p_opt_2, 
                                                               n=len(df[df['customer_type'] == 2]),
                                                                k=ki, epsilon=0.01, p=p_2, num_steps=num_steps,
                                                                global_min=context['min_price_2'], 
                                                                global_max=context['max_price_2'])

    # Update T
    T = T - (num_steps * xt)
    print("\nRemaining booking period 'T' = ", T, unit_T, "\n\n------------------------------------------------------------")

    if ki == K - 1:
        print("\n***Exploration finished***\n")
        print('Final values after Exploration:')
        print("Dual variable z* combined = ", z_opt)
        print(f"Optimal Prices: p1* = {p_opt_1:.2f}, p2* = {p_opt_2:.2f}")
        print("Remaining Inventory c = ", c)
        print("Remaining Booking Period T = ", T)
        print(f"Remaining inventory after sales: {c}, out of total inventory {c0}")
        print(f"Percentage of inventory remaining: {((c/c0)*100):.2f}%\n\n")
        print("---------------------------------------------------")
    return df, z_opt, (p_opt_1, p_opt_2), c, T, (p_low_1, p_low_2), (p_upp_1, p_upp_2), (new_price_list_1, new_price_list_2)


#################################################################################################################################################################################################
# (Logistic) - Demand function for Simulation for Exploration (P1)


# (Logistic) - Demand function - type  1: Reserve
"""Simulates demand using logistic function"""
def logistic_demand_1(pt, p, p0=0,L=1, k=0.009 ):
        p0=(p.min()+p.max())/2
        return L / (1 + np.exp(k * (pt - p0)))   
"""Returns the base probability of purchase at price p"""


# (Logistic) - Demand function - type  2: Preemtive
"""Simulates demand using logistic function"""
def logistic_demand_2(pt, p, p0=0,L=1, k=0.04 ):
        p0=(p.min()+p.max())/2
        return L / (1 + np.exp(k * (pt - p0)))   
"""Returns the base probability of purchase at price p"""



#####################################################################################################################################################################################################
# Compute Demand Probability Function - # Dx(p) - for Exploration (P1)

# Global dictionary to store empirical demand probabilities per price
demand_probability = {}

def compute_demand_probability(df, p):
    """
    Calculates empirical demand probabilities from exploration data.
    Uses 'p' list and 'pt' to access prices as per user's convention.
    Stores result in global variable `demand_probability`.
    """
    global demand_probability
    demand_probability = {}

    for pt in p:
        subset = df[df['price'] == pt]
        if not subset.empty:
            demand_prob = subset['buy'].sum() / subset['buy'].count()
            demand_probability[pt] = demand_prob
        else:
            demand_probability[pt] = 0.0  # fallback if no data at this price

    print("*** Empirical Demand Probabilities ***")
    for pt in p:
        print(f"Price: {pt:.2f}, Demand Probability: {demand_probability[pt]:.2f}")
    
    return demand_probability



################################################################################################################################################################################################
# Compute Revenue for each price for Exploration (P1)

def calculate_revenue(df):
    """
    Calculates total revenue from exploration sales data
    :param df: DataFrame containing sales data (must have 'price' and 'buy' columns)
    :return: Total revenue, Revenue per price point dictionary
    """
    # Calculate revenue for each transaction
    df['revenue'] = df['price'] * df['buy']
    
    # Calculate total revenue
    total_revenue = df['revenue'].sum()
    
    # Calculate revenue per price point
    revenue_per_price = df.groupby('price')['revenue'].sum().to_dict()
    
    return total_revenue, revenue_per_price


#######################################################################################################################################################################################################
# # Dual optimization: Find optimal value i.e. - z_opt in code and z* in paper for Exploration (P1)

"""For bounds from 0 to maximum price"""
""" Minimize_scalar used to optimize"""
# def dual_objective(z, demand_probability, p, c):
#     def objective(z):
#     # def objective(z_val):
#         return max((pt - z) * demand_probability[pt] for pt in p) + z * c

#     z_opt = minimize_scalar(objective, bounds=(0, max(p)), method='bounded')
#     return z_opt.x

""" For bounds from z (one unit of inventory - given by user at beginning) to maximum price"""
""" only Minimize used to optimize"""
def dual_objective(z, demand_probability, p, c):
    def objective(z):
        z = z[0]
        return (max((pt - z) * demand_probability[pt] for pt in p) + z * c)

    result = minimize(objective, x0=[z], bounds=[(z, max(p))])
    return result.x[0]


#######################################################################################################################################################################################################
# Calculate and find optimal price - p* in paper for Exploration (P1)

def exploration_optimal_price (z_opt, demand_probability, p):
    """
    Finds optimal price p* that maximizes (p - z*) * D_prob.(pt) (demand_probability[pt])
    where D_prob.(pt) is the empirical demand probability at price p.
    
    Parameters:
    - z_opt: optimal dual value (float)
    - demand_probability: dict mapping price -> demand probability
    - p: list or array of price points

    Returns:
    - p_opt: optimal price
    """
    optimal_price = None
    best_value = float('-inf')

    for pt in p:
        demand = demand_probability.get(pt, 0)
        value = (pt - float(z_opt)) * demand
        if value > best_value:
            best_value = value
            optimal_price = pt

    #print(f"*** Optimal Price Calculation ***")
    print(f"Optimal Price (p*): {optimal_price:.2f},\nFor Highest Revenue Value: {best_value*10:.2f}")
    return optimal_price


##########################################################################################################################################################################################
# Narrow Price Interval Function for Exploration

def narrow_price_interval(p_opt, n, k, epsilon, p, num_steps, global_min,global_max ):
    """
    Narrows the price interval around the estimated optimal price using 
    the theoretical shrinking factor from the paper.

    Parameters:
    - p_opt: Optimal price estimate from current subphase
    - n: Number of total customers observed so far
    - k: ki = Current subphase index (starting from 0)
    - epsilon: Small constant (e.g., 0.01)
    - p: Current price list (to extract full range)
    - num_steps: Number of price points to generate in next subphase

    Returns:
    - (p_low, p_upp, new_price_list): narrowed interval and refined grid
    """

    # Highest value of 10th for p_opt
    p_opt_index = np.where(p == p_opt)
    if p_opt==min(p):
        magnitude_p_opt = len(str(int(abs(p_opt))))
    else:
        magnitude_p_opt = len(str(int(abs(p[p_opt_index[0][0]-1]))))
    
    
    # Compute interval shrinking factor - parameter δ
    log_n = np.log(n) 
    delta = ((n ** ((-0.25) * (1 - ((3 / 5) ** k)))) * (log_n ** (-3 * epsilon)))  * (10**(magnitude_p_opt-1))
    #or
    #delta = ((n ** ((-0.25) * (1 - ((3 / 5) ** k)))) * (log_n ** (-3 * epsilon))+0.1)  * (10**(magnitude_p_opt-1))
    #or
    #delta = (n ** ((-0.25) * (1 - ((3 / 5) ** k))))*(10**(magnitude_p_opt-1))  #  * (log_n ** (-3 * epsilon)) 

    # Generate new interval around p_opt
    p_low = max(min(p), p_opt - delta)
    p_upp = min(max(p), p_opt + delta)

    # Ensure Interval does not Collapse*
    if p_low >= p_upp:
        # Fallback: Use 10% of current interval width
        width = (max(p) - min(p)) * 0.1
        p_low = max(global_min, p_opt - width)
        p_upp = min(global_max, p_opt + width)

    # New refined price grid
    new_price_list = np.linspace(p_low, p_upp, num_steps)

    #print(f"*** Subphase {k+1} Price Interval Update ***")
    print(f"delta: {delta:.5f}")
    print(f"New Interval: [{p_low:.2f}, {p_upp:.2f}]")
    #print(f"New Price List: {new_price_list}")
    #or
    print(f"New Price List: {[int(x) for x in new_price_list]}")


    return p_low, p_upp, new_price_list


#################################################################################################################################################################################
# Generate price ranges step wise as mentioned by user

# shared object just for num_steps: number of price points
# made this as num_steps is going to be used in two different functions
context = {}

# Generate price ranges step wise as mentioned by user
def generate_price_list_1(num_steps):
    min_price_1 = float(300)  #float(input("Enter the minimum price: "))
    max_price_1 = float(700)  #float(input("Enter the maximum price: "))
    num_steps = num_steps
    context['max_price_1'] = max_price_1
    context['min_price_1'] = min_price_1
    price_list = (np.linspace(min_price_1, max_price_1, num_steps))
    return price_list


# Generate price ranges step wise as mentioned by user
def generate_price_list_2(num_steps):
    min_price_2 = float(100)  #float(input("Enter the minimum price: "))
    max_price_2 = float(500)  #float(input("Enter the maximum price: "))
    num_steps = num_steps
    context['max_price_2'] = max_price_2
    context['min_price_2'] = min_price_2
    price_list = (np.linspace(min_price_2, max_price_2, num_steps))
    return price_list


##################################################################################################################################################################################
# Calculate how much unit of time is allocated to Phase 1 

def phase_one_days_calc(num_steps):
    xp=int(33)    #int(input("Enter % of Phase 1, between 0 to 100:"))
    num_steps = num_steps
    if (T*(xp/100)) % num_steps == 0:
        phase = T*xp
    else:
        phase = (T*(xp/100)) - ((T*(xp/100))%num_steps)
    return phase


####################################################################################################################################################################################
# Take input from the user
# or
# Give given data to code basically

c     = float(500) #float(input("Total Inventory 'c' :   "))
T     = float(100)  #float(input("Total booking peroid 'T' :  "))
unit_T= "days" #input("Enter unit of booking peroid (Usually days):")
t     = float(50)  #float(input("Instances 't' to divide Time peroid T into :  "))
xt    = T/t
num_steps = int(5)   #int(input("Enter how many price points you want: "))
p     = generate_price_list_1(num_steps)
p2    = generate_price_list_2(num_steps)
P1    = phase_one_days_calc(num_steps)   
K     = int (P1/(num_steps*xt))
z     = 80   #input("Enter imitial value of 1 unit of inventory (Can be zero): ")
customers_per_price = 10    #int(input("Enter number of customer approaching per price (e.g. 10):"))
total_customers = 2*customers_per_price*num_steps   #2 for 2 customer types 
c0    = c   #Inventory before or at start of booking period

print("*** USER INPUTS ***")
print("Total Inventory c:", c)
print("Total booking peroid T:", T,unit_T)
print("1 time Instance from t i.e. xt:", xt, unit_T)
print("Generated prices:", p)
print("Total time allocated to Phase 1:",P1)
print("Total subphases 'K' in Phase 1: ",K, '\n')

########################################################################################################################################################################
# Conduct Phase 1   i.e.  ***Exploration***: Call the function with user inputs as given above
# Or
# Phase 1 - Loop

print("\n","***Expoloration Start - Detailed Data:***")

for ki in range(K): 

    if c>0: # Inventory should exist  before sale
    
        print("\n","Phase 1 (P1), Subphase (ki): ",ki+1,"\n")
        
        df, z_opt, p_opt_tuple, c, T, p_low_tuple, p_upp_tuple, updated_p_tuple = exploration(c, c0,T, unit_T, t, xt, ki, K, P1, p, p2, z, total_customers, num_steps)
        
        # Update price lists for next iteration
        p = updated_p_tuple[0]
        p2 = updated_p_tuple[1]
        
    else:
        print("***NOT SUFFICIENT INVENTORY***")


############################################################################################################################################################################


*** USER INPUTS ***
Total Inventory c: 500.0
Total booking peroid T: 100.0 days
1 time Instance from t i.e. xt: 2.0 days
Generated prices: [300. 400. 500. 600. 700.]
Total time allocated to Phase 1: 30.0
Total subphases 'K' in Phase 1:  3 


 ***Expoloration Start - Detailed Data:***

 Phase 1 (P1), Subphase (ki):  1 

    customer_id  price  buy  customer_type
0             1  400.0    0              2
1             2  300.0    1              1
2             3  200.0    1              2
3             4  500.0    0              1
4             5  600.0    1              1
..          ...    ...  ...            ...
95           96  700.0    0              1
96           97  100.0    1              2
97           98  300.0    1              1
98           99  700.0    0              1
99          100  500.0    1              1

[100 rows x 4 columns]
Saved customer data to 'customer_demand_log.csv'

 Units sold in this phase: 49
Remaining inventory after sales: 451.0

** Type 1: Reserved

C:\Users\vijay\AppData\Local\Temp\ipykernel_9448\3655183692.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revenue'] = df['price'] * df['buy']
C:\Users\vijay\AppData\Local\Temp\ipykernel_9448\3655183692.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['revenue'] = df['price'] * df['buy']
C:\Users\vijay\AppData\Local\Temp\ipykernel_9448\3655183692.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va